# QuickTuneTool - A Demo


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/automl/QTT/blob/main/notebooks/demo.ipynb)

This guide will show you how to:

* Install `quicktunetool`,
* Use our meta-trained surrogate and start a run,
* Load and test a fine-tuned model for inference

## Install QTT and dependencies

In [ ]:
!pip install git+https://github.com/automl/QTT

## Download image dataset

To demonstrate the use of QTT, we will use the Imagenette dataset. Considering the limited hardware available on Colab, we will use only a small subset of the images.

- for *training* only 40 samples per class are used
- the validation set is split in *val* and *test*
- 10 samples per class for val
- 5 samples per class for test


In [ ]:
# @title download-script
%%capture
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
!tar -xzvf imagenette2-320.tgz

import os
import shutil


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def copy_files(src_dir, dest_dir, start_idx, end_idx):
    files = sorted(os.listdir(src_dir))
    total_files = len(files)
    if start_idx >= 0 and end_idx < total_files:
        for i in range(start_idx, end_idx + 1):
            shutil.copy(os.path.join(src_dir, files[i]), dest_dir)


# Create necessary directories
create_dir("dataset/train")
create_dir("dataset/val")
create_dir("dataset/test")

# Process training set
train_dir = "imagenette2-320/train"
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    if os.path.isdir(class_dir):
        create_dir(os.path.join("dataset/train", class_name))
        copy_files(class_dir, os.path.join("dataset/train", class_name), 0, 39)

# Process validation set
val_dir = "imagenette2-320/val"
for class_name in os.listdir(val_dir):
    class_dir = os.path.join(val_dir, class_name)
    if os.path.isdir(class_dir):
        create_dir(os.path.join("dataset/val", class_name))
        create_dir(os.path.join("dataset/test", class_name))

        total_files = len(os.listdir(class_dir))

        # Copy first 20 images to validation set
        copy_files(class_dir, os.path.join("dataset/val", class_name), 0, 19)

        # Copy last 5 images to test set
        copy_files(
            class_dir,
            os.path.join("dataset/test", class_name),
            total_files - 5,
            total_files - 1,
        )

## Setup

We utilize our predefined search space; however, similar to the image dataset, we adapt the hyperparameters to accommodate the limited hardware. We restrict the models to a maximum of 32 million parameters and reduce the maximum batch size to 32.

In [ ]:
import torch
from ConfigSpace.read_and_write import json as cs_json
from qtt.optimizers import QuickOptimizer
from qtt.tuner import QuickTuner

import warnings

warnings.filterwarnings("ignore")

In [ ]:
from ConfigSpace import (
    Categorical,
    ConfigurationSpace,
    Constant,
    EqualsCondition,
    OrConjunction,
    OrdinalHyperparameter,
)

cs = ConfigurationSpace("cv-classification/pipeline")

# finetuning parameters
freeze = OrdinalHyperparameter("pct_to_freeze", [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
ld = OrdinalHyperparameter("layer_decay", [0.0, 0.65, 0.75])
lp = OrdinalHyperparameter("linear_probing", [False, True])
sn = OrdinalHyperparameter("stoch_norm", [False, True])
sr = OrdinalHyperparameter("sp_reg", [0.0, 0.0001, 0.001, 0.01, 0.1])
d_reg = OrdinalHyperparameter("delta_reg", [0.0, 0.0001, 0.001, 0.01, 0.1])
bss = OrdinalHyperparameter("bss_reg", [0.0, 0.0001, 0.001, 0.01, 0.1])
cot = OrdinalHyperparameter("cotuning_reg", [0.0, 0.5, 1.0, 2.0, 4.0])

# regularization parameters
mix = OrdinalHyperparameter("mixup", [0.0, 0.2, 0.4, 1.0, 2.0, 4.0, 8.0])
mix_p = OrdinalHyperparameter("mixup_prob", [0.0, 0.25, 0.5, 0.75, 1.0])
cut = OrdinalHyperparameter("cutmix", [0.0, 0.1, 0.25, 0.5, 1.0, 2.0, 4.0])
drop = OrdinalHyperparameter("drop", [0.0, 0.1, 0.2, 0.3, 0.4])
smooth = OrdinalHyperparameter("smoothing", [0.0, 0.05, 0.1])
clip = OrdinalHyperparameter("clip_grad", [0, 1, 10])

# optimization parameters
amp = OrdinalHyperparameter("amp", [False, True])
opt = Categorical("opt", ["sgd", "momentum", "adam", "adamw", "adamp"])
betas = Categorical(
    "opt_betas", ["(0.9, 0.999)", "(0.0, 0.99)", "(0.9, 0.99)", "(0.0, 0.999)"]
)
lr = OrdinalHyperparameter("lr", [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01])
w_ep = OrdinalHyperparameter("warmup_epochs", [0, 5, 10])
w_lr = OrdinalHyperparameter("warmup_lr", [0.0, 1e-05, 1e-06])
wd = OrdinalHyperparameter("weight_decay", [0, 1e-05, 0.0001, 0.001, 0.01, 0.1])
bs = OrdinalHyperparameter("batch_size", [2, 4, 8, 16, 32, 64, 128, 256, 512])
mom = OrdinalHyperparameter("momentum", [0.0, 0.8, 0.9, 0.95, 0.99])
sched = Categorical("sched", ["cosine", "step", "multistep", "plateau"])
pe = OrdinalHyperparameter("patience_epochs", [2, 5, 10])
dr = OrdinalHyperparameter("decay_rate", [0.1, 0.5])
de = OrdinalHyperparameter("decay_epochs", [10, 20])
da = Categorical(
    "data_augmentation",
    ["auto_augment", "random_augment", "trivial_augment", "none"],
)
aa = Categorical("auto_augment", ["v0", "original"])
ra_nops = OrdinalHyperparameter("ra_num_ops", [2, 3])
ra_mag = OrdinalHyperparameter("ra_magnitude", [9, 17])
cond_1 = EqualsCondition(pe, sched, "plateau")
cond_2 = OrConjunction(
    EqualsCondition(dr, sched, "step"),
    EqualsCondition(dr, sched, "multistep"),
)
cond_3 = OrConjunction(
    EqualsCondition(de, sched, "step"),
    EqualsCondition(de, sched, "multistep"),
)
cond_4 = EqualsCondition(mom, opt, "momentum")
cond_5 = OrConjunction(
    EqualsCondition(betas, opt, "adam"),
    EqualsCondition(betas, opt, "adamw"),
    EqualsCondition(betas, opt, "adamp"),
)
cond_6 = EqualsCondition(ra_nops, da, "random_augment")
cond_7 = EqualsCondition(ra_mag, da, "random_augment")
cond_8 = EqualsCondition(aa, da, "auto_augment")
cs.add(
    mix,
    mix_p,
    cut,
    drop,
    smooth,
    clip,
    freeze,
    ld,
    lp,
    sn,
    sr,
    d_reg,
    bss,
    cot,
    amp,
    opt,
    betas,
    lr,
    w_ep,
    w_lr,
    wd,
    bs,
    mom,
    sched,
    pe,
    dr,
    de,
    da,
    aa,
    ra_nops,
    ra_mag,
    cond_1,
    cond_2,
    cond_3,
    cond_4,
    cond_5,
    cond_6,
    cond_7,
    cond_8,
)

# model
model = Categorical(
    "model",
    [
        "beit_base_patch16_384",
        "beit_large_patch16_512",
        "convnext_small_384_in22ft1k",
        "deit3_small_patch16_384_in21ft1k",
        "dla46x_c",
        "edgenext_small",
        "edgenext_x_small",
        "edgenext_xx_small",
        "mobilevit_xs",
        "mobilevit_xxs",
        "mobilevitv2_075",
        "swinv2_base_window12to24_192to384_22kft1k",
        "tf_efficientnet_b4_ns",
        "tf_efficientnet_b6_ns",
        "tf_efficientnet_b7_ns",
        "volo_d1_384",
        "volo_d3_448",
        "volo_d4_448",
        "volo_d5_448",
        "volo_d5_512",
        "xcit_nano_12_p8_384_dist",
        "xcit_small_12_p8_384_dist",
        "xcit_tiny_12_p8_384_dist",
        "xcit_tiny_24_p8_384_dist",
    ],
)
cs.add(model)

# max_fidelity
b = Constant("max_fidelity", 50)
cs.add(b)

### Load meta-trained surrogate model

The performance and cost predictors are metatrained on the learning curves of the meta-dataset.
With the predictors comes the configuration space (search-space), which we use to sample configurations.
And normalization parameters, that are the mean standard deviation of the training set of the meta-dataset.

We use a "ConfigManager" to sample random configurations and preprocess them for the optimization process.

In [ ]:
# load the metatrained predictors
perf_pred, cost_pred, cs, c_norm, m_norm = get_metatrained_surrogates("mtlbm/micro")


# create the optimizer
opt = QuickOptimizer(
    cs,  # configuration manager
    perf_pred,  # performance predictor
    cost_pred,  # cost predictor
    acq_fn="ei",  # acquisition function
    verbosity=2,  # verbosity level
)

## Run QuickTune
Specify the path and some other information of the dataset, that is given to the finetune-script.
Create the QuickTuner object, that handles the optimisation/finetuning process.

In the end, we get the best config and some information of process that we can save for later use.

In [ ]:
from qtt.finetune.cv.classification import finetune_script

qt = QuickTuner(opt, finetune_script)
task_info = {
    "data_path": "dataset",
    "train-split": "train",
    "val-split": "val",
    "num-classes": 10,
}
traj, runtime, history = qt.run(task_info=task_info, time_budget=600)

config_id, config, score, budget, cost = qt.get_incumbent()

## Plot the progress of the run

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

ids = history[:, 0]
scores = history[:, 2]

df_traj = pd.DataFrame({"score": traj, "runtime": runtime})
df_hist = pd.DataFrame({"id": ids, "score": scores, "runtime": runtime})
plt.figure(figsize=(10, 5))
sns.scatterplot(data=df_hist, x="runtime", y="score", hue="id", palette="tab10", s=100)
sns.lineplot(data=df_traj, x="runtime", y="score", color="red", label="Incumbent")
plt.title("QuickTuner - Trajectory / History")
plt.xlabel("Runtime (s)")
plt.ylabel("Score (%)")
plt.legend([], [], frameon=False)
plt.show()

In [ ]:
print("*****   QUICKTUNE RESULTS   *****")
print("=================================")
print()
print("Best configuration found:")
print(f"Archtitecture: {config['model']}")
print(f"Score: {score*100}")
print(f"Number of epochs trained: {budget}")
print(f"Cost per epoch: {cost}")
print(f"Config: {' '.join([f'{k}={v}' for k, v in config.items()])}")
print()
print("---------------------------------")
print()
print(f"Total number of evaluated configs: {len(qt.optimizer.evaluated_configs)}")
print(f"Total number of evaluations: {len(traj)}")

## Load fine-tuned model

We now load the fine-tuned model and test the performance on unseen data.

In [ ]:
import numpy as np
from timm.models import create_model
from torchvision import datasets
from torchvision import transforms as tf


def clean_state_dict(state_dict):
    state_dict = {k.replace("model._model.", ""): v for k, v in state_dict.items()}
    return state_dict


checkpoint_path = qt.output_path + f"/{config_id}/model_best.pth.tar"
# load checkpoint
checkpoint = torch.load(checkpoint_path, map_location="cpu")
state_dict = checkpoint["state_dict"]
state_dict = clean_state_dict(state_dict)

model_name = config["model"]
model = create_model(model_name, pretrained=False, num_classes=10)
model.load_state_dict(state_dict, strict=False)
model.eval()
model.to("cuda")

transform = tf.Compose(
    [
        tf.Resize(256),
        tf.CenterCrop(224),
        tf.ToTensor(),
        tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)
data_path = "dataset/test"
dataset = datasets.ImageFolder(data_path)

images = []
labels = []
for i in np.random.choice(len(dataset), size=10):
    img, label = dataset[i]
    images.append(img)
    labels.append(label)

input = torch.stack([transform(img) for img in images])
input = input.cuda()
with torch.no_grad():
    output = model(input)
    output = output.softmax(-1)
    _, indices = output.topk(1)

indices = indices.cpu().numpy().squeeze()

In [ ]:
import matplotlib.pyplot as plt

classes = [
    "trench",
    "english springer",
    "cassette player",
    "chain saw",
    "church",
    "french horn",
    "garbage truck",
    "gas pump",
    "golf ball",
    "parachute",
]


# Define a function to display a list of PIL images
def display_images(images, nrows=5, titles=None):
    n_images = len(images)
    if titles is None:
        titles = [""] * n_images
    fig = plt.figure(figsize=(4, 8))

    for n, (image, title) in enumerate(zip(images, titles)):
        ax = fig.add_subplot(nrows, int(n_images / nrows), n + 1)
        ax.imshow(image)
        ax.set_title(title)
        ax.axis("off")

    plt.tight_layout()
    plt.show()


titles = [classes[i] for i in indices]
display_images(images, titles=titles)